In [2]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pprint

from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import SLSQP, COBYLA, CG
from qiskit.chemistry import QiskitChemistryError
from qiskit.chemistry.applications import MolecularGroundStateEnergy
from qiskit.chemistry.algorithms import ground_state_solvers
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.core import QubitMappingType
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, HFMethodType

In [3]:
global qubit_mapping
qubit_mapping = 'jordan_wigner'
global two_qubit_reduction
two_qubit_reduction = False
global best_result
global hfmethod
hfmethod = HFMethodType.ROHF
global var_form
global vqe
global ferOp

In [4]:
backend = Aer.get_backend("statevector_simulator")
optimizer = COBYLA(maxiter=500)
distances = np.arange(0.6, 3.5, 0.1)
vqe_energies = []
min = 100.0
dismin = 0.0

for dist in distances:
    charge = 1
    spin = 0
    
    
    if dist <= 1.2:
        mol = "H .0 .0 .0; H .0 .0 " + str(dist) + "; H .0 .0 " + str(-dist)
    else:
        mol = "H .0 .0 .0; H .0 .0 " + str(dist) + "; H .0 .0 -0.742"
    
    driver = PySCFDriver(atom = mol,
                         unit=UnitsType.ANGSTROM, spin = spin,
                         charge=charge, basis='sto-6g',hf_method=hfmethod)
    
    molecule = driver.run()
    
    shift = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_orbitals = molecule.num_orbitals
    num_spin_orbitals = num_orbitals*2
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    
    qubitOp = ferOp.mapping(map_type=qubit_mapping)
    
    initial_state = HartreeFock(num_spin_orbitals, 
                                num_particles, 
                                qubit_mapping,
                                two_qubit_reduction)
    
    var_form = UCCSD(num_orbitals=num_spin_orbitals,
                        num_particles=num_particles,
                        initial_state=initial_state,
                        qubit_mapping=qubit_mapping,
                        two_qubit_reduction=two_qubit_reduction)
    
    init_parm = np.zeros(var_form.num_parameters)
    vqe = VQE(qubitOp, var_form, optimizer, initial_point=init_parm)
    vqe_result_tot = vqe.run(backend)
    vqe_result = np.real(vqe_result_tot['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    
    print("Interatomic Distance:", np.round(dist, 2), 
          "VQE Result:", vqe_result)
    if vqe_result < min:
        min = vqe_result
        dismin = dist
        best_result = vqe_result_tot

Interatomic Distance: 0.6 VQE Result: -1.126657479743669
Interatomic Distance: 0.7 VQE Result: -1.2059922365404094
Interatomic Distance: 0.8 VQE Result: -1.2375892574307057
Interatomic Distance: 0.9 VQE Result: -1.2423891811720058
Interatomic Distance: 1.0 VQE Result: -1.2319273697444038
Interatomic Distance: 1.1 VQE Result: -1.212919884183459
Interatomic Distance: 1.2 VQE Result: -1.1894591297206036
Interatomic Distance: 1.3 VQE Result: -1.2043268960869067
Interatomic Distance: 1.4 VQE Result: -1.194713510780188
Interatomic Distance: 1.5 VQE Result: -1.1857450678246018
Interatomic Distance: 1.6 VQE Result: -1.1777127298343995
Interatomic Distance: 1.7 VQE Result: -1.1707645748961912
Interatomic Distance: 1.8 VQE Result: -1.1649404564821948
Interatomic Distance: 1.9 VQE Result: -1.160198536999027
Interatomic Distance: 2.0 VQE Result: -1.1564402480014984
Interatomic Distance: 2.1 VQE Result: -1.1535335424509163
Interatomic Distance: 2.2 VQE Result: -1.1513338183757555
Interatomic Distan

In [9]:
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()
#plt.ylim([-1.3, -1])
plt.show()

In [8]:
distances = np.arange(0.3, 4.9, 0.1)

In [6]:
#da hartree a ev
num = 
print(num*13.6*2)

2.6299457278785594


In [3]:
#da ev a hartree
num = 
print(num/(13.6*2))

0.1875
